# Day 17: Volume Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class1_fundamentals/week4_charts_psychology/day17_volume.ipynb)

---

## Learning Objectives

1. Understand what volume represents in trading
2. Use volume to confirm price movements
3. Identify volume patterns and divergences
4. Apply On-Balance Volume (OBV) indicator

**Time**: 30 min lecture + 15 min hands-on

---

# Part 1: Lecture

---

## What Is Volume?

**Volume** = Total number of shares traded in a given period.

| Metric | Meaning |
|--------|--------|
| **High Volume** | Strong interest, conviction |
| **Low Volume** | Weak interest, less conviction |
| **Volume Spike** | Unusual activity, potential turning point |

### Why Volume Matters

```
Price moves WITHOUT volume = Suspect, may reverse
Price moves WITH volume = Confirmed, more likely to continue
```

> **Golden Rule:** "Volume precedes price" - Volume often signals moves before price confirms.

## Volume Confirmation

### Healthy Trends

| Scenario | Volume Should Be | Signal |
|----------|------------------|--------|
| **Uptrend** | Rising on up days | Bullish confirmation |
| **Downtrend** | Rising on down days | Bearish confirmation |
| **Breakout** | Spike on breakout day | Confirms breakout |
| **Reversal** | Spike at turning point | Confirms reversal |

### Warning Signs (Divergence)

| Price | Volume | Warning |
|-------|--------|--------|
| Rising | Falling | Uptrend weakening |
| Falling | Falling | Downtrend exhausting |
| New High | Lower than prev high | Potential top |
| New Low | Lower than prev low | Potential bottom |

## Volume Patterns

### 1. Volume Spike

```
Normal: ||| |||| ||| ||||
Spike:  ||| |||| ||||||||||| |||
                    ^
              Unusual activity
```

**Meaning:** Major news, institutional activity, or turning point.

### 2. Climax Volume

Extreme volume at the end of a trend often signals exhaustion:
- **Buying climax**: Huge volume at top = distribution
- **Selling climax**: Huge volume at bottom = capitulation

### 3. Dry Up

Volume declining before a move often precedes breakout.

## On-Balance Volume (OBV)

OBV is a cumulative volume indicator:

```
If Close > Previous Close: OBV = OBV + Volume
If Close < Previous Close: OBV = OBV - Volume
If Close = Previous Close: OBV = OBV
```

### OBV Interpretation

| OBV Trend | Price Trend | Signal |
|-----------|-------------|--------|
| Rising | Rising | Confirmed uptrend |
| Falling | Falling | Confirmed downtrend |
| Rising | Flat/Falling | Accumulation (bullish) |
| Falling | Flat/Rising | Distribution (bearish) |

## Key Concepts Summary

| Concept | Key Point |
|---------|----------|
| **Volume** | Confirms price moves |
| **Divergence** | Volume vs price = warning |
| **Spike** | Unusual activity signal |
| **Climax** | Extreme volume = exhaustion |
| **OBV** | Cumulative volume direction |

---

# Part 2: Hands-On

---

In [ ]:
# Setup
import sys
if 'google.colab' in sys.modules:
    !pip install yfinance pandas numpy matplotlib -q
    
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("Setup complete!")

In [ ]:
# Exercise 1: Volume Analysis Chart
ticker = 'AAPL'
data = yf.download(ticker, period='6mo', progress=False)

if not data.empty:
    # Calculate average volume
    data['Avg_Volume'] = data['Volume'].rolling(20).mean()
    data['Volume_Ratio'] = data['Volume'] / data['Avg_Volume']
    
    # Create chart
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 10), 
                                         gridspec_kw={'height_ratios': [3, 1, 1]})
    
    # Price
    ax1.plot(data.index, data['Close'], 'b-', linewidth=1)
    ax1.set_ylabel('Price ($)')
    ax1.set_title(f'{ticker} Price and Volume Analysis')
    ax1.grid(True, alpha=0.3)
    
    # Volume bars with color
    colors = ['green' if c >= o else 'red' 
              for c, o in zip(data['Close'], data['Open'])]
    ax2.bar(data.index, data['Volume'], color=colors, alpha=0.7)
    ax2.plot(data.index, data['Avg_Volume'], 'orange', linewidth=1, label='20-day Avg')
    ax2.set_ylabel('Volume')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Volume ratio
    ax3.bar(data.index, data['Volume_Ratio'], color='purple', alpha=0.5)
    ax3.axhline(y=1, color='black', linestyle='--')
    ax3.axhline(y=2, color='red', linestyle='--', label='2x Avg (spike)')
    ax3.set_ylabel('Vol/Avg')
    ax3.set_xlabel('Date')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Identify volume spikes
    spikes = data[data['Volume_Ratio'] > 2]
    print(f"\nVolume Spikes (>2x average): {len(spikes)}")

In [ ]:
# Exercise 2: On-Balance Volume (OBV)
def calculate_obv(data):
    """Calculate On-Balance Volume."""
    obv = [0]
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1]:
            obv.append(obv[-1] + data['Volume'].iloc[i])
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1]:
            obv.append(obv[-1] - data['Volume'].iloc[i])
        else:
            obv.append(obv[-1])
    return obv

if not data.empty:
    data['OBV'] = calculate_obv(data)
    
    # Plot OBV vs Price
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    ax1.plot(data.index, data['Close'], 'b-', linewidth=1)
    ax1.set_ylabel('Price ($)')
    ax1.set_title(f'{ticker} Price vs On-Balance Volume')
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(data.index, data['OBV'], 'purple', linewidth=1)
    ax2.set_ylabel('OBV')
    ax2.set_xlabel('Date')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Check for divergence
    price_trend = 'Up' if data['Close'].iloc[-1] > data['Close'].iloc[-30] else 'Down'
    obv_trend = 'Up' if data['OBV'].iloc[-1] > data['OBV'].iloc[-30] else 'Down'
    
    print(f"\n30-Day Trends:")
    print(f"Price: {price_trend}")
    print(f"OBV: {obv_trend}")
    if price_trend != obv_trend:
        print("** DIVERGENCE DETECTED - Potential reversal signal **")

---

# Part 3: Quiz

---

In [ ]:
questions = [
    {"q": "High volume on an up day confirms:", "a": "B",
     "opts": ["A) Weakness", "B) Bullish strength", "C) Reversal", "D) Nothing"]},
    {"q": "Price rising with falling volume suggests:", "a": "A",
     "opts": ["A) Uptrend weakening", "B) Strong uptrend", "C) Accumulation", "D) High conviction"]},
    {"q": "A volume spike often indicates:", "a": "C",
     "opts": ["A) Normal trading", "B) Low interest", "C) Potential turning point", "D) Market holiday"]},
    {"q": "OBV rising while price is flat suggests:", "a": "B",
     "opts": ["A) Distribution", "B) Accumulation", "C) No signal", "D) Sell signal"]},
]

print("Day 17 Quiz: Volume")
print("="*40)
for i, q in enumerate(questions, 1):
    print(f"\nQ{i}: {q['q']}")
    for opt in q['opts']:
        print(f"  {opt}")
    print(f"Answer: {q['a']}")

---

## Day 17 Summary

**Key Takeaways:**
1. Volume confirms price movements
2. Divergence between price and volume = warning
3. Volume spikes signal important activity
4. OBV tracks cumulative buying/selling pressure

**Next:** Day 18 - Timeframes

---
*Money Talks*